In [8]:
import json
from ete3 import Tree, TreeStyle, Tree, TextFace, add_face_to_node
import pandas_profiling as pf
import pandas as pd
with open('exp3_2.json', 'r') as openfile:
    json_object = json.load(openfile)

In [2]:
def getWorkerTree(worker):
    parent = {"children":[]}
    for result in worker['results']:
        if result['k'] <= 1:
            child = {"id" : "{}_{}_{}_{}".format(result['mazeEndIter'], result['workerDRCCost'], result['workerMarkerCost'], result['followGuide']).lower()}
            child['markers'] = result['markers']
            child['children'] = []
            child['k'] = result['k']
            child['tid'] = result['id']
            child['time'] = result['time']
            child['mazeSearchOps'] = result['mazeSearchOps']
            if result['k'] == 0:
                parent['children'].insert(0, child)
            else:
                parent['children'].append(child)
    for result in worker['results']:
        if result['k'] == 2:
            for item in parent['children']:
                if item['tid'] == result['parent']:
                    break
            child = {"id" : "{}_{}_{}_{}".format(result['mazeEndIter'], result['workerDRCCost'], result['workerMarkerCost'], result['followGuide']).lower()}
            child['markers'] = result['markers']
            child['children'] = []
            child['k'] = result['k']
            child['tid'] = result['id']
            child['time'] = result['time'] + item['time']
            child['mazeSearchOps'] = result['mazeSearchOps']
            item['children'].append(child)
    for result in worker['results']:
        if result['k'] == 3:
            found = False
            for chosen in parent['children']:
                for item in chosen['children']:
                    if item['tid'] == result['parent'] and item['markers'] != -1 and len(item['children']) < 5:
                        found = True
                        break
                if found:
                    break
            if not found:
                continue
            child = {"id" : "{}_{}_{}_{}".format(result['mazeEndIter'], result['workerDRCCost'], result['workerMarkerCost'], result['followGuide']).lower()}
            child['markers'] = result['markers']
            child['children'] = []
            child['k'] = result['k']
            child['time'] = result['time'] + item['time']
            child['mazeSearchOps'] = result['mazeSearchOps']
            item['children'].append(child)
    return parent

In [3]:
def addNode(node):
    name = "{}/{}".format(node['id'], node['markers'])
    treeStr = ""
    first = True
    for child in node['children']:
        childTreeStr = addNode(child)
        if first:
            treeStr += "("
        else:
            treeStr += ","
        treeStr += childTreeStr
        first = False
    if not first:
        treeStr += ")"
    treeStr += name
    return treeStr

In [ ]:
count = 0
for worker in json_object['tree']:
    if(worker['iter'] <= 59):
        continue
    print(worker['iter'])
    parent = getWorkerTree(worker)
    treeStr = "("
    first = True
    for child in parent['children']:
        if not first:
            treeStr += ","
        treeStr += addNode(child)
        first = False
    treeStr += ");"
    t = Tree( treeStr , format=1) 

    ts = TreeStyle()
    ts.show_leaf_name = False
    def my_layout(node):
            color = 'blue'
            if "-1" in node.name:
                color = 'red'
            if "/0" in node.name:
                color = 'green'
            F = TextFace(node.name, tight_text=True, fgcolor=color)
            add_face_to_node(F, node, column=0, position="branch-right")
    ts.layout_fn = my_layout
    ts.mode='c'
    t.show(tree_style=ts)
    count += 1
    if count > 5:
        break

60
60
61
61
61


In [4]:
def getBestChild(parent, ks):
    bestResult = {"markers":1e9, "mazeSearchOps":1e9, "time":1e9}
    for child in parent['children']:
        if child['markers'] == -1:
            continue
        ks[child['k']] += 1
        updateBest = False
        if child['markers'] < bestResult['markers']:
            updateBest = True
        elif child['markers'] == bestResult['markers'] and child['mazeSearchOps'] < bestResult['mazeSearchOps']:
            updateBest = True
        if updateBest:
            bestResult['markers'] = child['markers']
            bestResult['mazeSearchOps'] = child['mazeSearchOps']
            bestResult['time'] = child['time']
            bestResult['k'] = child['k']
        
        #check grandChildren
        updateBest = False
        bestChild = getBestChild(child, ks)
        if bestChild['markers'] < bestResult['markers']:
            updateBest = True
        elif bestChild['markers'] == bestResult['markers'] and bestChild['mazeSearchOps'] < bestResult['mazeSearchOps']:
            updateBest = True
        if updateBest:
            bestResult['markers'] = bestChild['markers']
            bestResult['mazeSearchOps'] = bestChild['mazeSearchOps']
            bestResult['time'] = bestChild['time']
            bestResult['k'] = bestChild['k']
    return bestResult
def getTotalMarkers(worker):
    markers = 0
    for tp in ['via', 'met']:
        for i in range(1,6):
            if tp == 'via' and i == 1:
                layerName = 'via'
            else:
                layerName = "{}{}".format(tp, i)
            if layerName not in worker:
                continue
            for col in worker[layerName]:
                if 'Constraint' in col:
                    markers += worker[layerName][col]
    return markers
allBest = []
for itr in range(0,64):
    maxTime = 0
    ks = [0,0,0,0]
    for worker in json_object['tree']:
        if(worker['iter'] != itr):
            continue
        parent = getWorkerTree(worker)
        bestResult = getBestChild(parent, ks)
        bestResult['committed'] = (bestResult['markers'] < getTotalMarkers(worker))
        bestResult['iter'] = itr
        maxTime = max(maxTime, bestResult['time'])
        allBest.append(bestResult)
#         print(bestResult)

    if maxTime == 0:
        continue
    print(itr, maxTime, ks)
#     x = input()

9 430 [33, 327, 1775, 2062]
13 143 [19, 148, 201, 190]
18 229 [9, 74, 0, 0]
22 522 [9, 74, 34, 10]
26 149 [10, 76, 136, 440]
30 481 [7, 58, 10, 6]
34 736 [8, 61, 10, 50]
38 1809 [6, 47, 10, 12]
42 1947 [6, 50, 76, 80]
46 199 [3, 21, 17, 2]
50 5019 [2, 19, 2, 0]
54 111 [2, 11, 4, 0]
58 124 [2, 20, 0, 0]
59 175 [2, 18, 3, 7]
60 178 [2, 20, 20, 20]
61 4121 [4, 39, 0, 0]
62 201 [1, 7, 0, 0]


In [5]:
df = pd.DataFrame(allBest)
rpt = pf.ProfileReport(df)
rpt

In [5]:
data = []
for worker in json_object['tree']:
    initMarkers = getTotalMarkers(worker)
    bestSet = False
    parent = getWorkerTree(worker)
    bestResult = getBestChild(parent, ks)
    originalMarkers = -1
    for result in worker['results']:
        if result['k'] == 0:
            originalMarkers = result['markers']
    
    for result in worker['results']:
        row = result.copy()
        row['better_than_init'] = (row['markers'] != -1 and row['markers'] < initMarkers )
        row['same_as_init'] = (row['markers'] != -1 and row['markers'] == initMarkers )
        row['better_than_original'] = (row['markers'] != -1 and row['markers'] < originalMarkers )
        row['same_as_original'] = (row['markers'] != -1 and row['markers'] == originalMarkers )
        row['iter'] = worker['iter']
        row['best'] = False
        row['timedout'] = row['markers'] == -1
        if not bestSet and bestResult['markers'] == row['markers'] and bestResult['k'] == row['k'] and bestResult['mazeSearchOps'] == row['mazeSearchOps']:
            row['best'] = True
            bestSet = True
        data.append(row)

In [9]:
import dtale
df = pd.DataFrame(data)
d = dtale.show(df)
d.open_browser()

Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/osama/.local/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/osama/.local/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "/home/osama/.local

2022-12-07 15:54:19,272 - ERROR    - Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/osama/.local/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/osama/.local/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback 

Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/osama/.local/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/osama/.local/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "/home/osama/.local

2022-12-07 15:54:29,557 - ERROR    - Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/osama/.local/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/osama/.local/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback 

Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/osama/.local/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/osama/.local/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "/home/osama/.local

...

2022-12-07 17:15:48,973 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/osama/.local/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/osama/.local/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError("Not running with the Werkzeug Server")
RuntimeError: Not running with the Werkzeug Server


2022-12-07 17:15:48,974 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/osama/.local/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/osama/.local/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError("Not running with the Werkzeug Server")
RuntimeError: Not runnin

Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/osama/.local/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/osama/.local/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "/home/osama/.local

2022-12-07 18:06:48,021 - ERROR    - Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/osama/.local/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/osama/.local/lib/python3.9/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/home/osama/.local/lib/python3.9/site-packages/dash/_callback.py", line 440, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback 

In [41]:
data

[{'mazeEndIter': 3,
  'workerDRCCost': 16,
  'workerMarkerCost': 1024,
  'followGuide': False,
  'markers': 3,
  'k': 1,
  'time': 12,
  'parent': '',
  'id': '3_16_1024_false',
  'mazeSearchOps': 15741350,
  'better_than_init': True,
  'same_as_init': False,
  'iter': 9,
  'best': False},
 {'mazeEndIter': 3,
  'workerDRCCost': 16,
  'workerMarkerCost': 2048,
  'followGuide': False,
  'markers': 3,
  'k': 1,
  'time': 15,
  'parent': '',
  'id': '3_16_2048_false',
  'mazeSearchOps': 20198440,
  'better_than_init': True,
  'same_as_init': False,
  'iter': 9,
  'best': False},
 {'mazeEndIter': 3,
  'workerDRCCost': 16,
  'workerMarkerCost': 2048,
  'followGuide': True,
  'markers': 12,
  'k': 1,
  'time': 22,
  'parent': '',
  'id': '3_16_2048_true',
  'mazeSearchOps': 33161015,
  'better_than_init': True,
  'same_as_init': False,
  'iter': 9,
  'best': False},
 {'mazeEndIter': 8,
  'workerDRCCost': 8,
  'workerMarkerCost': 32,
  'followGuide': False,
  'markers': 8,
  'k': 0,
  'time': 